In [49]:
#imports
import pandas as pd
import math
%matplotlib inline
import matplotlib.pyplot as plt
import bt
from bt.core import Algo
from pypfopt.expected_returns import mean_historical_return
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.risk_models import risk_matrix
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
import numpy as np
from math import sqrt

In [50]:
#import data
input_data = pd.read_excel("Assignments2021_QPM_corrected_09-12-2021.xlsm", sheet_name = "ETF & FF Factor Time Series")

In [51]:
#clean data
input_data = input_data.set_index("Date")

#adjust timeframe to start at July 18th 2013
input_data = input_data.truncate("2013-07-18")

In [52]:
#Creating separate df for the baseline portfolio

p_baseline = input_data.loc[:, ['iShares Russell 2000 ETF (IWM) $', 'iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) €', 'iShares MSCI Emerging Markets ETF (EEM) $', 'iShares Core Nikkei 225 ETF (1329.T) ¥', 'iShares 20+ Year Treasury Bond ETF (TLT) $', 'iShares Core U.S. Aggregate Bond ETF (AGG) $', 'SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $', 'iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $']]

p_baseline['iShares Core Nikkei 225 ETF (1329.T) ¥'] = p_baseline['iShares Core Nikkei 225 ETF (1329.T) ¥'] / input_data['USD/JPY (JPY=X) ¥']

p_baseline['iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) €'] = p_baseline['iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) €'] / input_data['USD/EUR (EUR=X) €']

p_baseline = p_baseline.rename(columns = {'iShares Core Nikkei 225 ETF (1329.T) ¥' : 'iShares Core Nikkei 225 ETF (1329.T) $', 'iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) €' : 'iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $'})

In [53]:
#Creating separate df for the benchmark portfolio 

p_benchmark = input_data.loc[:, ['Wilshire 5000 Total Market Index (^W5000) $']]

irx_yield = input_data.loc[:, '13 Week Treasury Bill (^IRX) $'] / 100

p_benchmark['Derived IRX Bond Price'] = 100 / (1 + irx_yield/4)

In [54]:
weights_60_40 = {
    "iShares Russell 2000 ETF (IWM) $"                            : 0.7*0.6,
    "iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $"         : 0.1*0.6,
    "iShares MSCI Emerging Markets ETF (EEM) $"                   : 0.1*0.6,
    "iShares Core Nikkei 225 ETF (1329.T) $"                      : 0.1*0.6,
    "iShares 20+ Year Treasury Bond ETF (TLT) $"                  : 0.1,
    "iShares Core U.S. Aggregate Bond ETF (AGG) $"                : 0.1,
    "SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $"         : 0.1,
    "iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $" : 0.1
}

weights_60_40_b = {
    "Wilshire 5000 Total Market Index (^W5000) $"                 : 0.6,
    "Derived IRX Bond Price"                                      : 0.4
}

data = p_baseline
data_b = p_benchmark

In [55]:
#creating the minimum volatility weight function to be used in the minvol algo
def calc_pypf_minvol_weights(prices):
    mu = mean_historical_return(prices, compounding= True)
    s  = risk_matrix(prices, method= "sample_cov")
    
    ef =  EfficientFrontier(mu, s)
    minvol_weights = ef.min_volatility()

    return minvol_weights

#create the weighminvol class to be used as an algo in the bt framework
class WeighMinVol(Algo):

    """
    Sets temp['weights'] based on minimum variance optimization.

    Sets the target weights based on calc_min_vol_weights.

    Args:
        * lookback (DateOffset): lookback period for estimating volatility
        * bounds ((min, max)): tuple specifying the min and max weights for
          each asset in the optimization.
        * covar_method (str): method used to estimate the covariance.
        * rf (float): risk-free rate used in optimization.

    Sets:
        * weights

    Requires:
        * selected
    """

    def __init__(
        self,
        lookback=pd.DateOffset(months=3),
        bounds=(0.0, 1.0),
        covar_method="ledoit-wolf",
        rf=0.0,
        lag=pd.DateOffset(days=0),
    ):
        super(WeighMinVol, self).__init__()
        self.lookback = lookback
        self.lag = lag
        self.bounds = bounds
        self.covar_method = covar_method
        self.rf = rf

    def __call__(self, target):
        selected = target.temp["selected"]

        if len(selected) == 0:
            target.temp["weights"] = {}
            return True

        if len(selected) == 1:
            target.temp["weights"] = {selected[0]: 1.0}
            return True

        t0 = target.now - self.lag
        prc = target.universe.loc[t0 - self.lookback : t0, selected]
        tw = calc_pypf_minvol_weights(prc)

        target.temp["weights"] = tw
        return True

In [56]:
#strategies
strategy_60_40 = bt.Strategy('Strategy 60/40', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights_60_40),
                    bt.algos.Rebalance()])

strategy_60_40_b = bt.Strategy('Strategy 60/40 benchmark', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights_60_40_b),
                    bt.algos.Rebalance()])

strategy_meanvar = bt.Strategy('Strategy Markowitz', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectHasData(),
                    bt.algos.WeighMeanVar(covar_method='standard'),
                    bt.algos.Rebalance()])

strategy_minvol = bt.Strategy('Strategy Minimum Volatility', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectHasData(),
                    WeighMinVol(covar_method='standard'),
                    bt.algos.Rebalance()])

strategy_1n =  bt.Strategy('Strategy 1/N', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighEqually(),
                    bt.algos.Rebalance()])

In [57]:
#1)

#60/40 portfolio
backtest_60_40 = bt.Backtest(strategy_60_40, data)
result_60_40 = bt.run(backtest_60_40)

#60/40 benchmark
backtest_60_40_b = bt.Backtest(strategy_60_40_b, data_b)
result_60_40_b = bt.run(backtest_60_40_b)

#Markowitz portfolio
backtest_meanvar = bt.Backtest(strategy_meanvar, data)
result_meanvar   = bt.run(backtest_meanvar)

#Minimum Volatility
backtest_minvol = bt.Backtest(strategy_minvol, data)
result_minvol   = bt.run(backtest_minvol)

#1/N portfolio
backtest_1n = bt.Backtest(strategy_1n, data)
result_1n   = bt.run(backtest_1n)

In [ ]:
#weights
weights_60_40 = result_60_40.get_security_weights()
weights_60_40_b = result_60_40_b.get_security_weights()
weights_meanvar = result_meanvar.get_security_weights()
weights_minvol = result_minvol.get_security_weights()
weights_1n = result_1n.get_security_weights()

weights_60_40, weights_60_40_b, weights_meanvar, weights_minvol, weights_1n

(            iShares Russell 2000 ETF (IWM) $  \
 2013-07-17                          0.000000   
 2013-07-18                          0.419912   
 2013-07-19                          0.420056   
 2013-07-22                          0.419625   
 2013-07-23                          0.418674   
 ...                                      ...   
 2021-08-26                          0.422294   
 2021-08-27                          0.421242   
 2021-08-28                          0.421242   
 2021-08-29                          0.421242   
 2021-08-30                          0.420955   
 
             iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $  \
 2013-07-17                                           0.000000     
 2013-07-18                                           0.059978     
 2013-07-19                                           0.059898     
 2013-07-22                                           0.060014     
 2013-07-23                                           0.059898     
 .

In [ ]:
#plotting the weights
fig, ax = plt.subplots(figsize=(30,15))

plt.subplot(3,2,1)
plt.plot(weights_60_40_b)
plt.title('Constant Weights Monthly Rebalancing Benchmark Portfolio')
plt.legend(labels = weights_60_40_b.columns, ncol=2, fancybox=True, bbox_to_anchor=(0.75, -0.1))

plt.subplot(3,2,2)
plt.plot(weights_60_40)
plt.title('Constant Weights Monthly Rebalancing Baseline Portfolio')
plt.legend(labels = weights_60_40.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.015, -0.1))

plt.subplot(3,2,3)
plt.plot(weights_minvol)
plt.title('MinVol Weights Monthly Rebalancing Baseline Portfolio')
plt.legend(labels = weights_minvol.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.015, -0.1))

plt.subplot(3,2,4)
plt.plot(weights_meanvar)
plt.title('MeanVar Weights Monthly Rebalancing Baseline Portfolio')
plt.legend(labels = weights_meanvar.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.015, -0.1))

plt.subplot(3,2,5)
plt.plot(weights_1n)
plt.title('1/N Weights Monthly Rebalancing Baseline Portfolio')
plt.legend(labels = weights_1n.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.015, -0.1))

plt.tight_layout()

In [ ]:
#plotting the return
result_60_40.plot()
result_60_40_b.plot()
result_meanvar.plot()
result_minvol.plot()
result_1n.plot()

<AxesSubplot:title={'center':'Equity Progression'}>

In [ ]:
result_1n.plot_histogram()
result_60_40.plot_histogram()
result_60_40_b.plot_histogram()
result_meanvar.plot_histogram()
result_minvol.plot_histogram()
result_1n.plot_histogram()

In [ ]:
result_1n.display_monthly_returns()
result_60_40.display_monthly_returns()
result_60_40_b.display_monthly_returns()
result_meanvar.display_monthly_returns()
result_minvol.display_monthly_returns()
result_1n.display_monthly_returns()

  Year    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2013   0      0      0      0      0      0     -0.87  -1.66   4.05   2.16   0.86   0.41   4.95
  2014  -1.32   2.43   0.34   0.13   1.81   1.52  -1.04   1.91  -3.01   1.74   0.26  -0.53   4.17
  2015   1.9    1.82   0.27   0.67  -0.45  -1.17  -0.54  -3.93  -1.81   3.29  -0.35  -1.86  -2.34
  2016  -2.79  -0.52   5.14   1.79  -0.05   1.18   3.15   0.99   0.42  -1.68  -1.49   1.5    7.63
  2017   1.64   1.25   0.91   1.68   1.66   0.79   1.42   0.56   1.12   1.6    0.85   1.03  15.49
  2018   2.02  -3.18   0.14   0.11   0.26  -0.75   1.52   0.22  -0.21  -5.53   1.15  -2.67  -6.96
  2019   5.49   0.94   1.52   1.53  -2.24   4.16  -0.28   0.7    0.85   1.77   0.9    2.1   18.67
  2020  -0.87  -2.87  -8.87   6.32   3.33   2.34   3.78   1.14  -1.95  -0.71   8.7    3.49  13.41
  2021  -0.31   0.39

In [ ]:
#2)
data_f = data
data_f["iShares Edge MSCI USA Momentum Factor ETF (MTUM) $"] = input_data["iShares Edge MSCI USA Momentum Factor ETF (MTUM) $"]
data_f["iShares Edge MSCI USA Size Factor ETF (SIZE) $"] = input_data["iShares Edge MSCI USA Size Factor ETF (SIZE) $"]
data_f["iShares Edge MSCI USA Value Factor ETF (VLUE) $"] = input_data["iShares Edge MSCI USA Value Factor ETF (VLUE) $"]
data_f["iShares Edge MSCI USA Quality Factor ETF (QUAL) $"] = input_data["iShares Edge MSCI USA Quality Factor ETF (QUAL) $"]
data_f["iShares Edge MSCI Min Vol USA ETF (USMV) $"] = input_data["iShares Edge MSCI Min Vol USA ETF (USMV) $"]
data_f = data_f.drop(columns="iShares Russell 2000 ETF (IWM) $")

weights_60_40_f = {
    "iShares Edge MSCI USA Momentum Factor ETF (MTUM) $"          : 0.2*0.7*0.6, 
    "iShares Edge MSCI USA Size Factor ETF (SIZE) $"              : 0.2*0.7*0.6,
    "iShares Edge MSCI USA Value Factor ETF (VLUE) $"             : 0.2*0.7*0.6,
    "iShares Edge MSCI USA Quality Factor ETF (QUAL) $"           : 0.2*0.7*0.6,
    "iShares Edge MSCI Min Vol USA ETF (USMV) $"                  : 0.2*0.7*0.6,
    "iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $"         : 0.1*0.6,
    "iShares MSCI Emerging Markets ETF (EEM) $"                   : 0.1*0.6,
    "iShares Core Nikkei 225 ETF (1329.T) $"                      : 0.1*0.6,
    "iShares 20+ Year Treasury Bond ETF (TLT) $"                  : 0.1,
    "iShares Core U.S. Aggregate Bond ETF (AGG) $"                : 0.1,
    "SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $"         : 0.1,
    "iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $" : 0.1
}

In [ ]:
#strategies
strategy_60_40_f = bt.Strategy('Strategy 60/40 with factors', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights_60_40_f),
                    bt.algos.Rebalance()])

strategy_meanvar_f = bt.Strategy('Strategy Markowitz with factors', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectHasData(),
                    bt.algos.WeighMeanVar(covar_method='standard'),
                    bt.algos.Rebalance()])

strategy_minvol_f = bt.Strategy('Strategy Minimum Volatility with factors', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectHasData(),
                    WeighMinVol(covar_method='standard'),
                    bt.algos.Rebalance()])

strategy_1n_f =  bt.Strategy('Strategy 1/N with factors', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighEqually(),
                    bt.algos.Rebalance()])

In [ ]:
data_f

,iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $,iShares MSCI Emerging Markets ETF (EEM) $,iShares Core Nikkei 225 ETF (1329.T) $,iShares 20+ Year Treasury Bond ETF (TLT) $,iShares Core U.S. Aggregate Bond ETF (AGG) $,SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $,iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $,iShares Edge MSCI USA Momentum Factor ETF (MTUM) $,iShares Edge MSCI USA Size Factor ETF (SIZE) $,iShares Edge MSCI USA Value Factor ETF (VLUE) $,iShares Edge MSCI USA Quality Factor ETF (QUAL) $,iShares Edge MSCI Min Vol USA ETF (USMV) $
Date,,,,,,,,,,,,
2013-07-18,32.074035,33.942856,144.681942,89.731071,89.689812,80.979561,90.054146,49.466301,46.467369,46.713112,44.641930,29.231804
2013-07-19,32.069039,33.848072,140.772565,91.150017,89.924026,80.939804,90.549377,49.685699,46.467369,46.627766,44.294781,29.300894
2013-07-22,32.232164,34.184132,142.295192,91.283585,90.041153,81.118759,90.753754,49.749687,47.308399,46.755787,44.508408,29.309526
2013-07-23,32.221520,34.554668,147.582860,90.899635,89.990952,80.880157,90.612259,49.667419,47.308399,46.755787,44.437202,29.274981
2013-07-24,32.483218,34.201370,144.437504,89.747780,89.698212,80.303612,90.172050,49.493721,47.308399,46.755787,44.374882,29.084999
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-26,55.133641,50.959999,259.693029,148.449997,115.779999,109.500000,134.630005,180.850006,132.889999,104.459999,139.789993,77.120003
2021-08-27,55.319774,51.590000,258.523560,149.460007,116.099998,109.860001,135.380005,183.350006,134.229996,105.529999,141.199997,77.459999
2021-08-28,55.319774,51.590000,258.523560,149.460007,116.099998,109.860001,135.380005,183.350006,134.229996,105.529999,141.199997,77.459999


In [ ]:
#60/40 with factors (fixed)
backtest_60_40_f = bt.Backtest(strategy_60_40_f, data_f)
result_60_40_f = bt.run(backtest_60_40_f)

#MSR with factors
backtest_meanvar_f = bt.Backtest(strategy_meanvar_f, data_f)
result_meanvar_f = bt.run(backtest_meanvar_f)

#MinVol with factors
backtest_minvol_f = bt.Backtest(strategy_minvol_f, data_f)
result_minvol_f = bt.run(backtest_minvol_f)

#1/N with factors
backtest_1n_f = bt.Backtest(strategy_1n_f, data_f)
result_1n_f = bt.run(backtest_1n_f)

In [ ]:
#weights
weights_60_40_f = result_60_40_f.get_security_weights()
weights_60_40_b = result_60_40_b.get_security_weights()
weights_meanvar_f = result_meanvar_f.get_security_weights()
weights_minvol_f = result_minvol_f.get_security_weights()
weights_1n_f = result_1n_f.get_security_weights()


In [ ]:
results_f = pd.DataFrame()
results = pd.DataFrame(index = result_60_40_f.stats.index)
results["Strategy 60/40 with factors"] = result_60_40_f.stats
results["Strategy 60/40 benchmark"] = result_60_40_b.stats
results["Strategy Markowitz with factors"] = result_meanvar_f.stats
results["Strategy MinVol with factors"] = result_minvol_f.stats
results["Strategy 1/N with factors"] = result_1n_f.stats

results

,Strategy 60/40 with factors,Strategy 60/40 benchmark,Strategy Markowitz with factors,Strategy MinVol with factors,Strategy 1/N with factors
start,2013-07-17 00:00:00,2013-07-17 00:00:00,2013-07-17 00:00:00,2013-07-17 00:00:00,2013-07-17 00:00:00
end,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00
rf,0.0,0.0,0.0,0.0,0.0
total_return,1.064546,0.820403,0.833246,0.339849,1.08021
cagr,0.093375,0.076561,0.077493,0.036684,0.094394
max_drawdown,-0.23649,-0.208878,-0.159877,-0.119637,-0.253274
calmar,0.394839,0.366532,0.484704,0.306628,0.372693
mtd,0.01228,0.015,0.013961,0.001495,0.012778
three_month,0.033597,0.038156,0.036043,0.012481,0.02854
six_month,0.082469,0.089689,0.11442,0.020599,0.079998


In [ ]:
#plotting the weights
fig, ax = plt.subplots(figsize=(30,15))

plt.subplot(3,2,1)
plt.plot(weights_60_40_f)
plt.title('Constant Weights Monthly Rebalancing Benchmark Factor Portfolio')
plt.legend(labels = weights_60_40_f.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.05, -0.1))

plt.subplot(3,2,2)
plt.plot(weights_60_40_b)
plt.title('Constant Weights Monthly Rebalancing Baseline Portfolio')
plt.legend(labels = weights_60_40_b.columns, ncol=3, fancybox=True, bbox_to_anchor=(0.85, -0.1))

plt.subplot(3,2,3)
plt.plot(weights_minvol_f)
plt.title('MinVol Weights Monthly Rebalancing Baseline Factor Portfolio')
plt.legend(labels = weights_minvol_f.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.05, -0.1))

plt.subplot(3,2,4)
plt.plot(weights_meanvar_f)
plt.title('MeanVar Weights Monthly Rebalancing Baseline Factor Portfolio')
plt.legend(labels = weights_meanvar_f.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.05, -0.1))

plt.subplot(3,2,5)
plt.plot(weights_1n_f)
plt.title('1/N Weights Monthly Rebalancing Baseline Factor Portfolio')
plt.legend(labels = weights_1n_f.columns, ncol=3, fancybox=True, bbox_to_anchor=(1.05, -0.1))

plt.tight_layout()


In [ ]:
#plotting the return
result_60_40_f.plot()
result_60_40_b.plot()
result_meanvar_f.plot()
result_minvol_f.plot()
result_1n_f.plot()


<AxesSubplot:title={'center':'Equity Progression'}>

In [ ]:
result_60_40_f.plot_histogram()
result_60_40_b.plot_histogram()
result_meanvar_f.plot_histogram()
result_minvol_f.plot_histogram()
result_1n_f.plot_histogram()

/Users/leftstories/Library/Python/3.8/lib/python/site-packages/ffn/core.py:691: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=figsize)


In [ ]:
#2c)
momentum_f = data_f['iShares Edge MSCI USA Momentum Factor ETF (MTUM) $']
value_f = data_f['iShares Edge MSCI USA Value Factor ETF (VLUE) $']

m_return = (momentum_f.pct_change() + 1).cumprod()
v_return = (value_f.pct_change()+1).cumprod()

plt.figure(figsize=(15,10))
plt.plot(m_return, label = 'Momentum ETF Total Return Factor')
plt.plot(v_return, label = 'Value ETF Total Return Factor')
plt.grid(alpha = 0.6)
plt.legend()
plt.xlabel('Date')
plt.ylabel('Total Return Factor')
plt.title('Momentum vs Value')

#Annualized return, volatility, sharpe ratio, maximum drawdown 
ar_m = mean_historical_return(momentum_f, compounding = True)[0]
ar_v = mean_historical_return(value_f, compounding = True)[0]

vola_m = momentum_f.pct_change().std() * sqrt(250)
vola_v= value_f.pct_change().std() * sqrt(250)
sr_m = ar_m / vola_m
sr_v = ar_v / vola_v

def max_dd(returns):
    """"
    Returns = df with the returns series 
    
    """
    r = returns.add(1).cumprod()
    dd = r.div(r.cummax()).sub(1)
    mdd = dd.min()
    end = dd.argmin()
    start = r.iloc[:end].argmax()
    return mdd, start, end

max_dd_m = max_dd(momentum_f.pct_change())[0]
max_dd_v = max_dd(value_f.pct_change())[0]

d = {'Annualised Return': [ar_m, ar_v], 'Volatility': [vola_m, vola_v], 'Sharpe Ratio': [sr_m, sr_v], 
     'Maximim Drawdown': [max_dd_m, max_dd_v]}
df = pd.DataFrame(data=d, index = ['Momentum', 'Value'])
df.T

/Library/Python/3.8/site-packages/pypfopt/expected_returns.py:123: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)


,Momentum,Value
Annualised Return,0.165807,0.100135
Volatility,0.185637,0.184701
Sharpe Ratio,0.893180,0.542149
Maximim Drawdown,-0.340823,-0.394705


In [ ]:
#Exercise 3
p_alternative = input_data.loc[:, ['iShares Russell 2000 ETF (IWM) $', 
'iShares Core U.S. REIT ETF (USRT) $', 
'IQ Hedge Multi-Strategy Tracker ETF (QAI) $', 
'iShares Global Infrastructure ETF (IGF) $', 
'Invesco Global Listed Private Equity ETF (PSP) $',
'iShares 20+ Year Treasury Bond ETF (TLT) $', 
'iShares Core U.S. Aggregate Bond ETF (AGG) $', 
'SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $', 
'iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $']]

weights_60_40_a = {
    "iShares Russell 2000 ETF (IWM) $"                            : 0.7*0.6,
    "iShares Core U.S. REIT ETF (USRT) $"                         : 0.3/4*0.6,
    "IQ Hedge Multi-Strategy Tracker ETF (QAI) $"                 : 0.3/4*0.6,
    "iShares Global Infrastructure ETF (IGF) $"                   : 0.3/4*0.6,
    "Invesco Global Listed Private Equity ETF (PSP) $"            : 0.3/4*0.6,
    "iShares 20+ Year Treasury Bond ETF (TLT) $"                  : 0.1,
    "iShares Core U.S. Aggregate Bond ETF (AGG) $"                : 0.1,
    "SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $"         : 0.1,
    "iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $" : 0.1
}

In [ ]:
strategy_60_40_a = bt.Strategy('Strategy 60/40 Alternative Assets', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectHasData(),
                    bt.algos.WeighSpecified(**weights_60_40_a),
                    bt.algos.Rebalance()])

backtest_60_40_a = bt.Backtest(strategy_60_40_a, p_alternative)
result_60_40_a = bt.run(backtest_60_40_a)

In [ ]:
results = pd.DataFrame(index = result_60_40.stats.index)
results['Benchmark 60/40'] = result_60_40_b.stats
results['Baseline 60/40'] = result_60_40.stats
results['Baseline 60/40 with Alternatives'] = result_60_40_a.stats
results


,Benchmark 60/40,Baseline 60/40,Baseline 60/40 with Alternatives
start,2013-07-17 00:00:00,2013-07-17 00:00:00,2013-07-17 00:00:00
end,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00
rf,0.0,0.0,0.0
total_return,0.820403,0.922889,0.927905
cagr,0.076561,0.083846,0.084194
max_drawdown,-0.208878,-0.274099,-0.281568
calmar,0.366532,0.305898,0.299019
mtd,0.015,0.00444,0.004579
three_month,0.038156,0.00498,0.017562
six_month,0.089689,0.026347,0.048413


In [ ]:
#Markowitz portfolio
backtest_meanvar_a = bt.Backtest(strategy_meanvar, p_alternative)
result_meanvar_a   = bt.run(backtest_meanvar_a)

#Minimum Volatility
backtest_minvol_a = bt.Backtest(strategy_minvol, p_alternative)
result_minvol_a   = bt.run(backtest_minvol_a)

#1/N portfolio
backtest_1n_a = bt.Backtest(strategy_1n, p_alternative)
result_1n_a   = bt.run(backtest_1n_a)

In [ ]:
results = pd.DataFrame(index = result_60_40.stats.index)
results['MeanVar 60/40 with Alternatives'] = result_meanvar_a.stats
results['MinVol 60/40 with Alternatives'] = result_minvol_a.stats
results['1/N with Alternatives'] = result_1n_a.stats 
results

,MeanVar 60/40 with Alternatives,MinVol 60/40 with Alternatives,1/N with Alternatives
start,2013-07-17 00:00:00,2013-07-17 00:00:00,2013-07-17 00:00:00
end,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00
rf,0.0,0.0,0.0
total_return,0.819898,0.227801,0.72299
cagr,0.076524,0.025595,0.069294
max_drawdown,-0.133474,-0.102723,-0.249816
calmar,0.573324,0.24916,0.277381
mtd,0.003069,0.001453,0.004913
three_month,0.035918,0.015764,0.032676
six_month,0.128954,0.019341,0.077254


In [ ]:
weights_60_40_a = result_60_40_a.get_security_weights()
weights_meanvar_a = result_meanvar_a.get_security_weights()
weights_minvol_a = result_minvol_a.get_security_weights()
weights_1n_a = result_1n_a.get_security_weights()

In [ ]:
#plotting the weights
fig, ax = plt.subplots(figsize=(30,15))

plt.subplot(3,2,1)
plt.plot(weights_60_40_a)
plt.title('Constant Weights Monthly Rebalancing Alternative Portfolio')
plt.legend(labels = weights_60_40_a.columns, ncol=2, fancybox=True, bbox_to_anchor=(0.75, -0.1))

plt.subplot(3,2,2)
plt.plot(weights_minvol_a)
plt.title('MinVol Weights Monthly Rebalancing Alternative Portfolio')
plt.legend(labels = weights_minvol_a.columns, ncol=3, fancybox=True, bbox_to_anchor=(1, -0.1))

plt.subplot(3,2,3)
plt.plot(weights_meanvar_a)
plt.title('MeanVar Weights Monthly Rebalancing Alternative Portfolio')
plt.legend(labels = weights_meanvar_a.columns, ncol=3, fancybox=True, bbox_to_anchor=(1, -0.1))

plt.subplot(3,2,4)
plt.plot(weights_1n_a)
plt.title('1/N Weights Monthly Rebalancing Alternative Portfolio')
plt.legend(labels = weights_1n_a.columns, ncol=3, fancybox=True, bbox_to_anchor=(1, -0.1))

plt.tight_layout()

In [ ]:
#Correlations
import numpy as np

#list with alts:
alt_list = [
p_alternative["IQ Hedge Multi-Strategy Tracker ETF (QAI) $"]      ,
p_alternative["iShares Global Infrastructure ETF (IGF) $"]    ,
p_alternative["Invesco Global Listed Private Equity ETF (PSP) $"],
p_alternative["iShares Core U.S. REIT ETF (USRT) $"]  ]

#correlation
for element in alt_list:
    print(np.corrcoef(element, data_b["Wilshire 5000 Total Market Index (^W5000) $"])[0,1])

0.9666074890943054
0.8328982889785433
0.9667521311503932
0.8582644635130361


In [ ]:
#Exercise 4 

#Deriving Expected Returns Matrix via Black - Litterman 

p_baseline_bl = p_baseline.loc[:, [
'iShares Russell 2000 ETF (IWM) $', 
'iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $',
'iShares MSCI Emerging Markets ETF (EEM) $',
'iShares Core Nikkei 225 ETF (1329.T) $',
'iShares 20+ Year Treasury Bond ETF (TLT) $',
'iShares Core U.S. Aggregate Bond ETF (AGG) $',
'SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $',
'iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $'

]]

mu_1 = mean_historical_return(p_baseline_bl.loc['2013-07-18':'2019-12-31'], compounding = True)
s_1 = risk_matrix(p_baseline_bl.loc['2013-07-18':'2019-12-31'], method="sample_cov")

viewdict = {
'iShares Russell 2000 ETF (IWM) $': 0.089377 - 0.01, 
'iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $': 0.057931 + 0.01,
'iShares 20+ Year Treasury Bond ETF (TLT) $': 0.064232 - 0.005
}

confidences = [0.6, 0.6, 0.75]

bl = BlackLittermanModel(s_1, pi=mu_1, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

ret_bl = bl.bl_returns()

rets_df = pd.DataFrame([mu_1, bl.bl_returns(), pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
iShares Russell 2000 ETF (IWM) $,0.089377,0.085179,0.079377
iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $,0.057931,0.063190,0.067931
iShares MSCI Emerging Markets ETF (EEM) $,0.043294,0.041288,NaN
iShares Core Nikkei 225 ETF (1329.T) $,0.069625,0.070826,NaN
iShares 20+ Year Treasury Bond ETF (TLT) $,0.064232,0.060788,0.059232
iShares Core U.S. Aggregate Bond ETF (AGG) $,0.033101,0.032201,NaN
SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $,0.041952,0.041127,NaN
iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $,0.052839,0.051354,NaN


In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
ef = EfficientFrontier(ret_bl, S_bl)
ef.max_sharpe()
weights = ef.clean_weights()

In [ ]:
strategy_bl = bt.Strategy('Black Litterman', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights),
                    bt.algos.Rebalance()])

backtest_bl = bt.Backtest(strategy_bl, p_baseline_bl.loc['2020-01-01':'2020-12-31'])
result_bl = bt.run(backtest_bl)
result_bl.stats

,Black Litterman
start,2020-01-01 00:00:00
end,2020-12-31 00:00:00
rf,0.0
total_return,0.114129
cagr,0.114212
max_drawdown,-0.234165
calmar,0.487741
mtd,0.017892
three_month,0.084893
six_month,0.084696


In [ ]:
#Exercise 4 b
mu_1 = mean_historical_return(data_f.loc['2013-07-18':'2019-12-31'], compounding = True)
s_1 = risk_matrix(data_f.loc['2013-07-18':'2019-12-31'], method="sample_cov")

viewdict = {
'iShares Edge MSCI USA Value Factor ETF (VLUE) $': 0.103631 + 0.02, 
'iShares 20+ Year Treasury Bond ETF (TLT) $': 0.064232 - 0.005
}

confidences = [0.7, 0.75]

bl = BlackLittermanModel(s_1, pi=mu_1, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

ret_bl = bl.bl_returns()

rets_df = pd.DataFrame([mu_1, bl.bl_returns(), pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
iShares STOXX Europe 600 UCITS ETF (DE) (EXSA.DE) $,0.057931,0.065088,NaN
iShares MSCI Emerging Markets ETF (EEM) $,0.043294,0.056094,NaN
iShares Core Nikkei 225 ETF (1329.T) $,0.069625,0.072775,NaN
iShares 20+ Year Treasury Bond ETF (TLT) $,0.064232,0.059344,0.059232
iShares Core U.S. Aggregate Bond ETF (AGG) $,0.033101,0.032241,NaN
SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $,0.041952,0.045690,NaN
iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $,0.052839,0.052234,NaN
iShares Edge MSCI USA Momentum Factor ETF (MTUM) $,0.154110,0.165569,NaN
iShares Edge MSCI USA Size Factor ETF (SIZE) $,0.119827,0.129790,NaN
iShares Edge MSCI USA Value Factor ETF (VLUE) $,0.103631,0.117676,0.123631


In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
ef = EfficientFrontier(ret_bl, S_bl)
ef.max_sharpe()
weights = ef.clean_weights()

In [ ]:
strategy_bl = bt.Strategy('Black Litterman', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights),
                    bt.algos.Rebalance()])

backtest_bl = bt.Backtest(strategy_bl, data_f.loc['2020-01-01':'2020-12-31'])
result_bl = bt.run(backtest_bl)
result_bl.stats

,Black Litterman
start,2020-01-01 00:00:00
end,2020-12-31 00:00:00
rf,0.0
total_return,0.099659
cagr,0.09973
max_drawdown,-0.227015
calmar,0.439311
mtd,0.015295
three_month,0.062063
six_month,0.085013


In [ ]:
mu_1 = mean_historical_return(p_alternative.loc['2013-07-18':'2019-12-31'], compounding = True)
s_1 = risk_matrix(p_alternative.loc['2013-07-18':'2019-12-31'], method="sample_cov")

viewdict = {
'iShares Core U.S. REIT ETF (USRT) $': 0.081466 - 0.01, 
'iShares Global Infrastructure ETF (IGF) $': 0.075719 + 0.01
}

confidences = [0.5, 1]

bl = BlackLittermanModel(s_1, pi=mu_1, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

ret_bl = bl.bl_returns()

rets_df = pd.DataFrame([mu_1, bl.bl_returns(), pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
iShares Russell 2000 ETF (IWM) $,0.089377,0.096121,NaN
iShares Core U.S. REIT ETF (USRT) $,0.081466,0.081484,0.071466
IQ Hedge Multi-Strategy Tracker ETF (QAI) $,0.024526,0.026790,NaN
iShares Global Infrastructure ETF (IGF) $,0.075719,0.085719,0.085719
Invesco Global Listed Private Equity ETF (PSP) $,0.089946,0.098602,NaN
iShares 20+ Year Treasury Bond ETF (TLT) $,0.064232,0.061695,NaN
iShares Core U.S. Aggregate Bond ETF (AGG) $,0.033101,0.032847,NaN
SPDR Bloomberg Barclays High Yield Bond ETF (JNK) $,0.041952,0.044795,NaN
iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD) $,0.052839,0.052764,NaN


In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
ef = EfficientFrontier(ret_bl, S_bl)
ef.max_sharpe()
weights = ef.clean_weights()

In [ ]:
strategy_bl = bt.Strategy('Black Litterman', [
                    bt.algos.RunMonthly(run_on_first_date=True, run_on_end_of_period=False),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights),
                    bt.algos.Rebalance()])

backtest_bl = bt.Backtest(strategy_bl, p_alternative.loc['2020-01-01':'2020-12-31'])
result_bl = bt.run(backtest_bl)
result_bl.stats

,Black Litterman
start,2020-01-01 00:00:00
end,2020-12-31 00:00:00
rf,0.0
total_return,0.09522
cagr,0.095288
max_drawdown,-0.253371
calmar,0.376081
mtd,0.021272
three_month,0.087011
six_month,0.077934
